In [2]:
!pip install -q tensorflow numpy pandas nltk keras
print('done!')

done!


In [3]:
import pandas as pd
import copy

# importa dataset
mental_health_conversation_dataset = pd.read_csv('./datasets/Mental_Health_FAQ.csv')
mental_health_conversation_dataset.drop(columns=['Question_ID'], axis=0, inplace=True)
# importa dataset
QA_python_dataset = pd.read_csv('./datasets/Dataset_Python_Question_Answer.csv')
QA_python_dataset = QA_python_dataset.rename(columns={"Question": "Questions", "Answer": "Answers"})
#importa dataset
conversation_3d_chatbot = pd.read_csv('./datasets/3d-conversation-chatbot.csv')
conversation_3d_chatbot.drop(columns=['Unnamed: 0'], axis=0, inplace=True)
conversation_3d_chatbot = conversation_3d_chatbot.rename(columns={"question": "Questions", "answer": "Answers"})
#importa dataset
# file = open("./datasets/jungleb.txt", "r")
# the_jungle_book = file.read()
# file.close()
# junta datasets
df_concated = pd.concat([mental_health_conversation_dataset, QA_python_dataset, conversation_3d_chatbot], ignore_index=True)
# tira dados vazios
df_concated.dropna()
# reseta indexes
df_concated = df_concated.reset_index(drop=True)
# clona dataset para conservar tratamento
df_clone = copy.deepcopy(df_concated)
# junta colunas
data = {"text": []}
# data['text'].append(the_jungle_book)
for x in range(0, len(df_clone)):
    text = "".join(df_clone['Questions'][x]+" "+df_clone['Answers'][x])
    data['text'].append(text) 
df_merged = pd.DataFrame(data)
df_clone2 = df_merged.copy()
print('done!')

done!


In [69]:
text = df_clone2.text.values
joined_text = "".join(text)
print('done!')

done!


In [82]:
joined_text = joined_text[:500000]

In [74]:
import random

import numpy as np
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop
print('done!')

done!


In [85]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(joined_text.lower())
tokens[:100]

['what',
 'does',
 'it',
 'mean',
 'to',
 'have',
 'a',
 'mental',
 'illness',
 'mental',
 'illnesses',
 'are',
 'health',
 'conditions',
 'that',
 'disrupt',
 'a',
 'personâ',
 's',
 'thoughts',
 'emotions',
 'relationships',
 'and',
 'daily',
 'functioning',
 'they',
 'are',
 'associated',
 'with',
 'distress',
 'and',
 'diminished',
 'capacity',
 'to',
 'engage',
 'in',
 'the',
 'ordinary',
 'activities',
 'of',
 'daily',
 'life',
 'mental',
 'illnesses',
 'fall',
 'along',
 'a',
 'continuum',
 'of',
 'severity',
 'some',
 'are',
 'fairly',
 'mild',
 'and',
 'only',
 'interfere',
 'with',
 'some',
 'aspects',
 'of',
 'life',
 'such',
 'as',
 'certain',
 'phobias',
 'on',
 'the',
 'other',
 'end',
 'of',
 'the',
 'spectrum',
 'lie',
 'serious',
 'mental',
 'illnesses',
 'which',
 'result',
 'in',
 'major',
 'functional',
 'impairment',
 'and',
 'interference',
 'with',
 'daily',
 'life',
 'these',
 'include',
 'such',
 'disorders',
 'as',
 'major',
 'depression',
 'schizophrenia',
 '

In [133]:
# unique_tokens = np.unique(tokens)
unique_tokens = list(set(tokens))
unique_tokens_idx = {token: idx for idx, token in enumerate(unique_tokens)} # achei o erro. token n era o nome da posicao, 
#era pra ser o valor mesmo
unique_tokens_idx

{'counsel': 0,
 'made': 1,
 'team': 2,
 'usually': 3,
 'author': 4,
 'beginning': 5,
 'property': 6,
 'stop': 7,
 'gmail': 8,
 'skills': 9,
 'traps': 10,
 'nutrition': 11,
 'available': 12,
 'eating': 13,
 'delusions': 14,
 'pancreas': 15,
 'discount': 16,
 'readers': 17,
 'education': 18,
 'helped': 19,
 'goal': 20,
 'where': 21,
 'focusing': 22,
 'missed': 23,
 'resource': 24,
 'hard': 25,
 'aren': 26,
 'vol14': 27,
 'occupation': 28,
 'return': 29,
 'multiple': 30,
 'work': 31,
 'so': 32,
 'bring': 33,
 'find': 34,
 'problems': 35,
 'alone': 36,
 'tool': 37,
 'treating': 38,
 'articles': 39,
 'possible': 40,
 'point': 41,
 'develop': 42,
 'major': 43,
 'virus': 44,
 'over': 45,
 'healthnewsreview': 46,
 'boring': 47,
 'problem': 48,
 'relationship': 49,
 'canada': 50,
 'talk': 51,
 'room': 52,
 'identification': 53,
 'keep': 54,
 'skill': 55,
 'method': 56,
 'actually': 57,
 'exercises': 58,
 'denial': 59,
 'wods': 60,
 'depending': 61,
 'capacity': 62,
 'parks': 63,
 'any': 64,
 'b

In [134]:
# separando dados para treino. pegamos todas as palavras mas 10 palavras a frente
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens)- n_words):
    input_words.append(tokens[i:i+n_words])
    next_words.append(tokens[i+n_words])

print('done!')

done!


In [135]:
# bag of words
# para cada exemplo, pegamos n_words a frente e depois o boolean para cada possível próxima palavra
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [136]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_tokens_idx[word]] = 1
    y[i, unique_tokens_idx[next_words[i]]] = 1

In [137]:
model = tf.keras.models.Sequential()
model.add(LSTM(128, input_shape = (n_words, len(unique_tokens)), return_sequences = True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))
print('done!')

done!


In [138]:
model.compile(loss="categorical_crossentropy", optimizer = RMSprop(learning_rate=0.01), metrics = ["accuracy"])
model.fit(X, y, batch_size = 128, epochs = 10, shuffle = True)

Epoch 1/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 9s 84ms/step - accuracy: 0.0222 - loss: 6.5988
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 83ms/step - accuracy: 0.0301 - loss: 6.1824
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 84ms/step - accuracy: 0.0307 - loss: 6.1135
Epoch 4/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 85ms/step - accuracy: 0.0341 - loss: 6.0578
Epoch 5/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 92ms/step - accuracy: 0.0454 - loss: 5.9014
Epoch 6/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 87ms/step - accuracy: 0.0541 - loss: 5.7452
Epoch 7/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 88ms/step - accuracy: 0.0742 - loss: 5.4913
Epoch 8/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 88ms/step - accuracy: 0.1014 - loss: 5.1933
Epoch 9/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 94ms/step - accuracy: 0.1186 - loss: 4.9329
Epoch 10/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 82ms/step - accuracy: 0.1735 - loss: 4.4946


In [139]:
# salva um arquivo com o modelo para poder ser recuperado depois
model.save("fourth-try-model.h5")
print('done!')

done!


In [140]:
model = load_model("fourth-try-model.h5")

In [141]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_tokens_idx[word]] = 1
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best]
print('done!')

done!


In [147]:
possible = predict_next_word("What am I doing", 5)
possible

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


783

In [143]:
def generate_text(input_text, text_length, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)
print('done!')

done!


In [144]:
generate_text("what can we do about AI ?", 100, 4)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━

'what can we do about AI ? program similar dosage weddings caretakers my whether hearing qualify reach spending organization regardless confused seeing appears comfort away pay agree seems actions learning youth committed future problemsolving 604 developing museums recover effectiveness sucks committed galleries stupid scared estimated capture barre week prevent field actions articles resources his call few art grocery role sooner both closures member articles antidepressant severity vegetables hear other however functional irregularly both examination listen provider effort map assumptions program encouraging safest hbr next friend interfere 4 deal college child psychological made include situations major recommendations posts felt problem his 310 bccdc read clean crossfit depending links'

In [ ]:
# As palavras não são encontradas na bag of words

In [ ]:
# Da respostas completamente desconexas mas acho q isso acontece pois a func predict_next_word não funciona corretamente